In [1]:
%matplotlib tk

In [2]:
import numpy as np
import pylab as pl
import pandas as pd

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM

In [6]:
df =  pd.read_csv("Champagne Sales.csv")
df.head()

,Month,Champagne sales
0,1964-01,2815
1,1964-02,2672
2,1964-03,2755
3,1964-04,2721
4,1964-05,2946


In [9]:
df.values

array([['1964-01', 2815],
       ['1964-02', 2672],
       ['1964-03', 2755],
       ['1964-04', 2721],
       ['1964-05', 2946],
       ['1964-06', 3036],
       ['1964-07', 2282],
       ['1964-08', 2212],
       ['1964-09', 2922],
       ['1964-10', 4301],
       ['1964-11', 5764],
       ['1964-12', 7312],
       ['1965-01', 2541],
       ['1965-02', 2475],
       ['1965-03', 3031],
       ['1965-04', 3266],
       ['1965-05', 3776],
       ['1965-06', 3230],
       ['1965-07', 3028],
       ['1965-08', 1759],
       ['1965-09', 3595],
       ['1965-10', 4474],
       ['1965-11', 6838],
       ['1965-12', 8357],
       ['1966-01', 3113],
       ['1966-02', 3006],
       ['1966-03', 4047],
       ['1966-04', 3523],
       ['1966-05', 3937],
       ['1966-06', 3986],
       ['1966-07', 3260],
       ['1966-08', 1573],
       ['1966-09', 3528],
       ['1966-10', 5211],
       ['1966-11', 7614],
       ['1966-12', 9254],
       ['1967-01', 5375],
       ['1967-02', 3088],
       ['196

In [10]:
pl.plot(df.values[:,1])

In [11]:
A = df['Champagne sales'].values
Scaling = A.max()
A = A / Scaling

In [15]:
A.shape[0]

105

In [17]:
#Prepare RNN Dataset. 
#Each data point (X) is linked to the previous data points of size=lookback
#The predicted value (Y) is the next point

lookback = 20

def create_rnn_dataset(data, lookback=1):
    
    data_x, data_y = [], []
    #for i in range(len(data) - lookback -1):
    for i in range(len(data[:,0]) - lookback -1):
        #All points from this point, looking backwards upto lookback
            a = data[i:(i+ lookback), :]
            data_x.append(a)
            #The next point
            data_y.append(data[i + lookback, :])
    return np.array(data_x), np.array(data_y)

#Create X and Y for training
train_req_x, train_req_y = create_rnn_dataset(A.reshape(A.shape[0],1) ,lookback)

#Reshape for use with LSTM
train_req_x = np.reshape(train_req_x,(train_req_x.shape[0],1, train_req_x.shape[1]))

print("Shapes of X, Y: ",train_req_x.shape, train_req_y.shape)

Shapes of X, Y:  (84, 1, 20) (84, 1)


In [18]:
#Create a Keras Model
ts_model=Sequential()
#Add LSTM
ts_model.add(LSTM(128, input_shape=(1,lookback)))
ts_model.add(Dense(1))

#Compile with Adam Optimizer. Optimize for minimum mean square error
ts_model.compile(loss="mean_squared_error",
                 optimizer="adam",
                 metrics=["mse"])

#Print model summary
ts_model.summary()

#Train the model
history = ts_model.fit(train_req_x,train_req_y, epochs=20, batch_size=1, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               76288     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 76,417
Trainable params: 76,417
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
84/84 [==============================] - 2s 2ms/step - loss: 0.0521 - mse: 0.0521
Epoch 2/20
84/84 [==============================] - 0s 2ms/step - loss: 0.0264 - mse: 0.0264 
Epoch 3/20
84/84 [==============================] - 0s 2ms/step - loss: 0.0194 - mse: 0.0194
Epoch 4/20
84/84 [==============================] - 0s 2ms/step - loss: 0.0118 - mse: 0.0118 
Epoch 5/20
84/84 [==============================] - 0s 2ms/step - loss: 0.

In [19]:
pl.figure(1)

loss = history.history['loss']
epochs = range(1, len(loss) + 1)
pl.plot(epochs, loss, "r--o", lw = 1, mfc = "none" ,label='Training loss')

pl.title('Training loss')
pl.xlabel('Epochs')
pl.ylabel('Loss')
pl.legend()
pl.show()

In [20]:
predict_on_train = ts_model.predict(train_req_x)

In [21]:
pl.figure(1)

pl.plot(predict_on_train*Scaling)
pl.plot(train_req_y*Scaling)

In [24]:
#predict in the future

#Predict for the next week
predict_for = 100

curr_input = train_req_x[-1,:,:].T 

for i in range(predict_for):
    
    #Take the last lookback no. of samples as X
    this_input_ = curr_input[-lookback:,:]
    #Create the input
    this_input = this_input_.reshape((1,1,lookback))
    #Predict for the next point
    this_prediction = ts_model.predict(this_input)

    #Add the current prediction to the input
    curr_input = np.vstack([curr_input,this_prediction])

#Extract the last predict_for part of curr_input, which contains all the new predictions
predict_on_future=np.reshape(np.array(curr_input[-predict_for:,:]),(predict_for,1))

In [25]:
nt  = train_req_x.shape[0]
Np  = predict_on_future.shape[0]

pl.figure(3)

pl.plot(range(0,nt),train_req_y[:,0]*Scaling)
pl.plot(range(nt,nt+Np),predict_on_future[:,0]*Scaling)